Loading and clearing the dataset

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_excel('edited master copy.xlsx')
dataset = dataset[dataset['sex_upon_intake'].notna()]


##taking intake age and intake number
X = dataset.iloc[:, [1,4,5,6,7,9,11,13]].values
y = dataset.iloc[:, 14].values


# Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)








In [59]:


len(dataset)

47802

Since most of the features provided are categorical in nature , one hot coding, label encoding has been performed on the respective cols

In [0]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer
labelcoder_y = LabelEncoder()
y = labelcoder_y.fit_transform(y)








In [61]:
y

array([6, 6, 6, ..., 6, 0, 3])

In [62]:
le_name_mapping = dict(zip(labelcoder_y.classes_, labelcoder_y.transform(labelcoder_y.classes_)))
print(le_name_mapping)

{'Adoption': 0, 'Died': 1, 'Disposal': 2, 'Euthanasia': 3, 'Missing': 4, 'Relocate': 5, 'Return to Owner': 6, 'Rto-Adopt': 7, 'Transfer': 8}


In [63]:
y[0:10]

array([6, 6, 6, 4, 6, 6, 6, 6, 6, 6])

In [64]:
X[100, :]


array(['Dog', 'Aged', 'Stray', 'Spayed Female', 14.0, 1, 'Spayed Female',
       5112.231944], dtype=object)

In [0]:
ct = ColumnTransformer([("encoding", OneHotEncoder(), [0,1,2,3,6])], remainder = 'passthrough')
X = ct.fit_transform(X)

In [66]:
X[100, :]

<1x30 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

 Splitting the dataset into the Training set and Test set

In [0]:

#from sklearn.cross_validation import train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [68]:
X_train

<38241x30 sparse matrix of type '<class 'numpy.float64'>'
	with 305750 stored elements in Compressed Sparse Row format>

In [69]:
y_train

array([5, 0, 1, ..., 0, 7, 0])

In [73]:
from xgboost import XGBClassifier
classifier1 = XGBClassifier()
print(classifier1)
 
# Cross validation scores
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier1, X = X_train, y = y_train, cv = 5)
#







XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [74]:
print("accuracies = ",accuracies," Mean accuracies score = ",np.mean(accuracies))

accuracies =  [0.57432344 0.56289226 0.57479079 0.57204498 0.57583682]  Mean accuracies score =  0.5719776589058323


In [0]:
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score
# Create parameters to search
params = {
     'learning_rate': [0.01],
     'n_estimators': [900,1000,1100],
     'max_depth':[7,8,9],
     'reg_alpha':[0.3,0.4,0.5]
    }
 
# Initializing the XGBoost Regressor
xgb_model = XGBClassifier()
 
# Gridsearch initializaation
gsearch = GridSearchCV(xgb_model, params,
                    verbose=True,
                    cv=5,
                    n_jobs=2)
 
gsearch.fit(X_train, y_train)
 
#Printing the best chosen params
print("Best Parameters :",gsearch.best_params_)
 
params = {'objective':'binary:logistic', 'booster':'gbtree'}
 
# Updating the parameter as per grid search
params.update(gsearch.best_params_)
 
# Initializing the XGBoost Regressor
xgb_model = xgb.XGBClassifier(**params)
print(xgb_model)
 
# Cross validation scores
# Cross validation scores
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier1, X = X_train, y = y_train, cv = 5)
print("accuracies = ",accuracies," Mean accuracies score = ",np.mean(accuracies))
 


Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [0]:
# Fitting model on tuned parameters
xgb_model.fit(X_train, y_train)

In [0]:
classifier1.fit(X_train, y_train)

In [0]:
y_pred = classifier1.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm_xgboost = confusion_matrix(y_test, y_pred)
cm_xgboost


array([[2203,    0,    0,   17,    0,    0,  312,    0,   75],
       [ 318,    0,    0,   28,    0,    0,   66,    0,  170],
       [   0,    0,    7,   26,    0,    0,    1,    0,    0],
       [  62,    1,    1,  516,    0,    0,   82,    0,  127],
       [ 348,    1,    0,    8,    0,    0,   67,    0,  133],
       [ 298,    0,    0,    7,    0,    0,   80,    0,  139],
       [ 271,    0,    0,   25,    0,    0, 1588,    0,   44],
       [ 282,    0,    0,    7,    0,    0,   64,    0,  120],
       [ 612,    1,    0,   51,    0,    0,  236,    0, 1167]])

In [16]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier1, X = X_train, y = y_train, cv = 5)
accuracies.mean()


0.5719776589058323

Over sampling of minority classes can help to create a better classifier . In the next section both oversampling and undersampling has been tried through a pipeline


In [17]:
print("Before OverSampling, counts of label '0': {}".format(sum(y_train == 0))) 
print("Before OverSampling, counts of label '1': {} \n".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '2': {} \n".format(sum(y_train == 2))) 
print("Before OverSampling, counts of label '3': {} \n".format(sum(y_train == 3))) 
print("Before OverSampling, counts of label '4': {} \n".format(sum(y_train == 4))) 
print("Before OverSampling, counts of label '5': {} \n".format(sum(y_train == 5))) 
print("Before OverSampling, counts of label '6': {} \n".format(sum(y_train == 6))) 
print("Before OverSampling, counts of label '7': {} \n".format(sum(y_train == 7))) 
print("Before OverSampling, counts of label '8': {} \n".format(sum(y_train == 8))) 









Before OverSampling, counts of label '0': 10360
Before OverSampling, counts of label '1': 2247 

Before OverSampling, counts of label '2': 144 

Before OverSampling, counts of label '3': 3355 

Before OverSampling, counts of label '4': 2042 

Before OverSampling, counts of label '5': 2049 

Before OverSampling, counts of label '6': 7789 

Before OverSampling, counts of label '7': 1977 

Before OverSampling, counts of label '8': 8278 



In [47]:
###oversampling using SMOTE
from imblearn.over_sampling import SMOTE
#dict1 = {2:2000 , 4:4000 , 5:4000 , 7:4000 }
smote = SMOTE(sampling_strategy='minority')
#smote = SMOTE(sampling_strategy=dict1)
X_sm, y_sm = smote.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
from imblearn.under_sampling import RandomUnderSampler


In [48]:
X_sm.shape

(46029, 30)

In [49]:
print("After OverSampling, counts of label '0': {}".format(sum(y_sm == 0))) 
print("After OverSampling, counts of label '1': {}".format(sum(y_sm == 1))) 
print("After OverSampling, counts of label '2': {}".format(sum(y_sm == 2))) 
print("After OverSampling, counts of label '3': {}".format(sum(y_sm == 3))) 
print("After OverSampling, counts of label '4': {}".format(sum(y_sm == 4))) 
print("After OverSampling, counts of label '5': {}".format(sum(y_sm == 5))) 
print("After OverSampling, counts of label '6': {}".format(sum(y_sm == 6))) 
print("After OverSampling, counts of label '7': {}".format(sum(y_sm == 7))) 
print("After OverSampling, counts of label '8': {}".format(sum(y_sm == 8))) 




After OverSampling, counts of label '0': 10360
After OverSampling, counts of label '1': 2247
After OverSampling, counts of label '2': 2000
After OverSampling, counts of label '3': 3355
After OverSampling, counts of label '4': 4000
After OverSampling, counts of label '5': 4000
After OverSampling, counts of label '6': 7789
After OverSampling, counts of label '7': 4000
After OverSampling, counts of label '8': 8278


In [50]:
from xgboost import XGBClassifier
classifier2 = XGBClassifier()
classifier2.fit(X_sm, y_sm)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred2 = classifier2.predict(X_test)

In [52]:
from sklearn.metrics import confusion_matrix
cm_xgboost = confusion_matrix(y_test, y_pred2)
cm_xgboost


array([[2191,    0,    1,   19,    0,    0,  318,    0,   78],
       [ 320,    0,    7,   21,    0,    0,   64,    0,  170],
       [   0,    0,   20,   13,    0,    0,    1,    0,    0],
       [  66,    0,   89,  422,    0,    0,   78,    0,  134],
       [ 348,    0,    0,   10,    0,    0,   67,    0,  132],
       [ 302,    0,    0,    8,    0,    0,   75,    0,  139],
       [ 338,    0,    1,   24,    0,    0, 1501,    0,   64],
       [ 282,    0,    0,    7,    0,    0,   64,    0,  120],
       [ 624,    0,    1,   71,    0,    0,  222,    0, 1149]])

In [53]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier2, X = X_sm, y = y_sm, cv = 5)
accuracies.mean()


0.5187606835539205

A trial with under sampling

In [0]:
# decision tree  on imbalanced dataset with SMOTE oversampling and random undersampling
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [57]:
# define dataset
X_train, y_train = X_train, y_train
# define pipeline
model = XGBClassifier()
over = SMOTE(sampling_strategy='minority')
#under = RandomUnderSampler(sampling_strategy='majority')
#steps = [('under', under), ('model', model)]
steps = [('over', over), ('model', model)]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X_train, y_train, scoring='f1_micro', cv=cv, n_jobs=-1)
print('Mean f1_scores: %.3f' % mean(scores))

Mean f1_scores: 0.415


Accuracy has increased to 63.6% when minority was sampled



mean f1 score .54 with only minority oversampling

Decreased further when all smaller samples were over sampled 

After applying both oversampling and then undersampling on majority class and majority class respectively f1 score decreased to .44



#  Output file generation/Prediction

We will be using our trained Xg boost classifier to predict the test data set and generate the output file. 

In [0]:
test_dataset = pd.read_excel('edited master test.xlsx')
X_test1 = test_dataset.iloc[:, [1,4,5,6,7,9,11,13]].values

In [0]:
len(test_dataset)

31869

In [0]:
X_test1[2,:]

<1x30 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [0]:
ct_test = ColumnTransformer([("encoding", OneHotEncoder(), [0,1,2,3,6])], remainder = 'passthrough')
X_test1 = ct_test.fit_transform(X_test1)

In [0]:
X_test1[2,:]

<1x30 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [0]:
y_pred1 = classifier2.predict(X_test1)

In [0]:
y_pred1[0:50]

array([3, 0, 0, 3, 3, 6, 3, 3, 0, 3, 8, 3, 0, 3, 3, 8, 3, 3, 0, 6, 3, 3,
       6, 8, 3, 6, 3, 3, 6, 3, 3, 3, 6, 6, 8, 6, 6, 8, 8, 8, 3, 3, 3, 6,
       8, 8, 6, 3, 3, 3])

In [0]:
y_pred1 = labelcoder_y.inverse_transform(y_pred1)

In [0]:
y_pred1

array(['Euthanasia', 'Adoption', 'Adoption', ..., 'Euthanasia',
       'Euthanasia', 'Euthanasia'], dtype=object)

In [0]:
output_file = test_dataset

In [0]:
output_file['outcome_type'] = y_pred1 

In [0]:
output_file['outcome_type']

0        Euthanasia
1          Adoption
2          Adoption
3        Euthanasia
4        Euthanasia
            ...    
31864      Transfer
31865    Euthanasia
31866    Euthanasia
31867    Euthanasia
31868    Euthanasia
Name: outcome_type, Length: 31869, dtype: object

In [0]:
output_file

In [0]:
output_file_final = output_file[['animal_id_outcome','outcome_type']]

In [0]:
output_file_final.to_csv('output_file_minority_oversampled.csv',index=False)